In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
ls

drive/  sample_data/


In [3]:
%cd /content/drive/MyDrive/DL_project/llm_finetuning/notebooks/

/content/drive/MyDrive/DL_project/llm_finetuning/notebooks


In [4]:
!pip install -q transformers accelerate bitsandbytes datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 55.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 28.1 MB/s eta 0:00:00


In [5]:
import torch
import numpy as np
from torch.nn.functional import kl_div, softmax, log_softmax
from torch.optim import AdamW
from datasets import load_dataset, ClassLabel
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss
from tqdm import tqdm


torch.cuda.empty_cache()

# for reproducibility
np.random.seed(42)

torch.manual_seed(42)

if torch.cuda.is_available():
  torch.cuda.manual_seed_all(42)


data = load_dataset("glue", "mnli")
hans_data = load_dataset("hans")


#Below method is refenced from: https://github.com/uds-lsv/llmft/blob/main/notebooks/majority_baseline.ipynb
def binarize_mnli(dataset, remove_neutral=True):
    if remove_neutral:
        # neutral class has label 1
        dataset = dataset.filter(lambda example: example["label"] != 1)

    # change labels of contradiction examples from 2 to 1
    def change_label(example):
        # convert labels 2 into labels 1. this merges the neutral and contradiction class
        example["label"] = 1 if example["label"] == 2 else example["label"]
        return example

    # change labels
    dataset = dataset.map(change_label)

    # change features to reflect the new labels
    features = dataset["train"].features.copy()
    features["label"] = ClassLabel(num_classes=2, names=['entailment', 'contradiction'], id=None)
    dataset = dataset.cast(features)  # overwrite old features

    return dataset

data = binarize_mnli(data, remove_neutral=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating validation_matched split:   0%|          | 0/9815 [00:00<?, ? examples/s]

Generating validation_mismatched split:   0%|          | 0/9832 [00:00<?, ? examples/s]

Generating test_matched split:   0%|          | 0/9796 [00:00<?, ? examples/s]

...

Casting the dataset:   0%|          | 0/261802 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/6692 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/6703 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/9796 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/9847 [00:00<?, ? examples/s]

In [6]:
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification, AutoTokenizer, AdamW, AutoConfig

import pandas as pd
from torch.nn import KLDivLoss

#function for computing accuracy
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy_score(labels, predictions)}

tokenizer = AutoTokenizer.from_pretrained("facebook/opt-125m")

# Define the fixed context
fixed_context = "Given the premise, does the hypothesis hold true? "

##Prepare the inputs with the fixed context
def manipulate_inputs(batch):
    encoding = tokenizer([f'{fixed_context} Premise: {premise} Hypothesis: {hypothesis}'
                          for premise, hypothesis in zip(batch["premise"], batch["hypothesis"])],
                          truncation=True, padding="max_length", max_length=128, return_tensors='pt')
    batch["input_ids"] = encoding["input_ids"].squeeze()
    batch["attention_mask"] = encoding["attention_mask"].squeeze()
    return batch

data = data.map(manipulate_inputs, batched=True)
hans_data = hans_data.map(manipulate_inputs, batched=True)

# Change num_labels to 2 and drop-out hyperparam = 0.1
config = AutoConfig.from_pretrained("facebook/opt-125m", num_labels=2, hidden_dropout_prob=0.1, attention_probs_dropout_prob=0.1)


# Define a KL divergence loss function
def custom_loss(model_probs, original_model_probs):
    return kl_div(model_probs.log_softmax(dim=-1), original_model_probs, reduction='batchmean')
task_loss = CrossEntropyLoss()



results_df = pd.DataFrame(columns=["n", "run", "in_domain_accuracy", "out_of_domain_accuracy"])
n_values = [2, 16, 32, 64, 128]  # number of examples for each class

for n in n_values:
    for run in range(1):  # repeat 10 times for each n
        # Data selection code remains the same
        indices_yes = np.where(np.array(data["train"]["label"]) == 0)[0]
        indices_no = np.where(np.array(data["train"]["label"]) == 1)[0]
        indices_yes = np.random.choice(indices_yes, n, replace=False)
        indices_no = np.random.choice(indices_no, n, replace=False)
        indices = np.concatenate([indices_yes, indices_no])
        train_dataset = data["train"].select(indices)
        print(train_dataset.shape)

        # Initialize both teacher and student models
        teacher_model = AutoModelForSequenceClassification.from_pretrained("facebook/opt-125m", config=config)
        student_model = AutoModelForSequenceClassification.from_pretrained("facebook/opt-125m", config=config)
        teacher_model.to(device)
        student_model.to(device)
        teacher_model.eval()  # Ensure the teacher model does not train

        # Training configuration
        total_steps = (len(train_dataset) // 32) * 40
        training_args = TrainingArguments(
            output_dir="./offload_folder",
            overwrite_output_dir=True,
            num_train_epochs=40,
            per_device_train_batch_size=32,
            learning_rate=1e-5,
            weight_decay=0.0,
            save_steps=10_000,
            save_total_limit=2,
            warmup_steps=int(0.1 * total_steps),
        )

        # Customize training loop to include KL divergence loss
        class CustomTrainer(Trainer):
            def compute_loss(self, model, inputs, return_outputs=False):
              # Getting the output from the model, which includes logits
              outputs = model(**inputs)
              student_logits = outputs.logits  # Correctly accessing the logits

              # Extract labels from the inputs for use in cross entropy loss
              labels = inputs["labels"]

              # Calculate the teacher model's logits without gradient updates
              with torch.no_grad():
                  teacher_outputs = teacher_model(**inputs)
                  teacher_logits = teacher_outputs.logits

              # Calculate the distillation loss using the custom KL divergence function
              distillation_loss = custom_loss(student_logits, teacher_logits)

              # Calculate the classification loss using cross entropy
              classification_loss = task_loss(student_logits, labels)

              # Combine the losses
              loss = 0.5 * distillation_loss + 0.5 * classification_loss
              return (loss, outputs) if return_outputs else loss

        # Define the trainer with the custom loss function
        trainer = CustomTrainer(
            model=student_model,
            args=training_args,
            train_dataset=train_dataset,
            compute_metrics=compute_metrics,
        )

        # Training, evaluation, and result collection code remains mostly unchanged
        trainer.train()

        print(f"Evaluating in-domain performance for n={n}...")
        eval_results = trainer.evaluate(eval_dataset=data["validation_matched"])
        in_domain_accuracy = eval_results["eval_accuracy"]
        for key, value in eval_results.items():
            print(f"In-domain {key}: {value}")

        print(f"Evaluating out-of-domain performance for n={n}...")
        eval_results = trainer.evaluate(eval_dataset=hans_data["validation"])
        out_of_domain_accuracy = eval_results["eval_accuracy"]
        for key, value in eval_results.items():
            print(f"Out-of-domain {key}: {value}")

        new_row = pd.DataFrame({
            "n": [n],
            "run": [run],
            "in_domain_accuracy": [in_domain_accuracy],
            "out_of_domain_accuracy": [out_of_domain_accuracy]
        })
        results_df = pd.concat([results_df, new_row], ignore_index=True)
        print(results_df)

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/651 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

...

In-domain eval_loss: nan
In-domain eval_accuracy: 0.5853257621040048
In-domain eval_runtime: 155.0922
In-domain eval_samples_per_second: 43.149
In-domain eval_steps_per_second: 5.397
In-domain epoch: 40.0
Evaluating out-of-domain performance for n=64...
Out-of-domain eval_loss: nan
Out-of-domain eval_accuracy: 0.49673333333333336
Out-of-domain eval_runtime: 671.9816
Out-of-domain eval_samples_per_second: 44.644
Out-of-domain eval_steps_per_second: 5.581
Out-of-domain epoch: 40.0
    n run  in_domain_accuracy  out_of_domain_accuracy
0   2   0            0.480126                0.500000
1  16   0            0.544979                0.500000
2  32   0            0.544680                0.500000
3  64   0            0.585326                0.496733
(256, 6)


Some weights of OPTForSequenceClassification were not initialized from the model checkpoint at facebook/opt-125m and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of OPTForSequenceClassification were not initialized from the model checkpoint at facebook/opt-125m and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss


Evaluating in-domain performance for n=128...


In-domain eval_loss: nan
In-domain eval_accuracy: 0.6708009563658099
In-domain eval_runtime: 158.1832
In-domain eval_samples_per_second: 42.305
In-domain eval_steps_per_second: 5.291
In-domain epoch: 40.0
Evaluating out-of-domain performance for n=128...
Out-of-domain eval_loss: nan
Out-of-domain eval_accuracy: 0.5
Out-of-domain eval_runtime: 676.5026
Out-of-domain eval_samples_per_second: 44.346
Out-of-domain eval_steps_per_second: 5.543
Out-of-domain epoch: 40.0
     n run  in_domain_accuracy  out_of_domain_accuracy
0    2   0            0.480126                0.500000
1   16   0            0.544979                0.500000
2   32   0            0.544680                0.500000
3   64   0            0.585326                0.496733
4  128   0            0.670801                0.500000


In [7]:
!ls

few_shot_context_distillation_mnli_baseline_results.csv  results
logs							 vanilla_mnli_baseline_results.csv
models							 vanilla_mnli_baseline_results_hans_350.csv
offload_folder


In [8]:
results_df.to_csv("./few_shot_context_distillation_mnli_baseline_results.csv", index=False)

In [9]:
results_df

,n,run,in_domain_accuracy,out_of_domain_accuracy
0,2,0,0.480126,0.500000
1,16,0,0.544979,0.500000
2,32,0,0.544680,0.500000
3,64,0,0.585326,0.496733
4,128,0,0.670801,0.500000
